<a href="https://colab.research.google.com/github/Garvit1721/Text_Generation_using_LSTM/blob/main/text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://www.gutenberg.org/files/1661/1661-0.txt -O book.txt

--2024-05-09 06:41:11--  https://www.gutenberg.org/files/1661/1661-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607504 (593K) [text/plain]
Saving to: ‘book.txt’

book.txt            100%[===================>] 593.27K  1.94MB/s    in 0.3s    

2024-05-09 06:41:12 (1.94 MB/s) - ‘book.txt’ saved [607504/607504]



In [ ]:
text = open('book.txt','r',encoding='utf-8').read()
text = text.lower()

In [ ]:
sentences = text.split('\n')

In [ ]:
import tensorflow as tf
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences


In [ ]:
tokenizer = Tokenizer(oov_token='<UNK>')

In [ ]:
tokenizer.fit_on_texts(sentences)

In [ ]:
vocab_size = len(tokenizer.word_index)
vocab_size

8922

In [ ]:
sequences = tokenizer.texts_to_sequences(sentences)

In [ ]:
input_sequences = []
for sequence in sequences:
  for i in range(1,len(sequence)):
    n_gram_sequence = sequence[:i+1]
    input_sequences.append(n_gram_sequence)

In [ ]:
print(input_sequences[0],input_sequences[1])

[4776, 158] [4776, 158, 331]


In [ ]:
max_seq_len = max([len(seq) for seq in input_sequences])
max_seq_len

20

In [ ]:
padded_sequences = pad_sequences(sequences,maxlen = max_seq_len)

In [ ]:
print((padded_sequences[0]),(padded_sequences[1]))

[   0    0    0    0    0    0    0    0    0    0 4776  158  331  886
    5    2 1092    5  130   35] [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0   46  656 2664 2665]


In [ ]:
import numpy as np
padded_sequences = np.array(padded_sequences)

In [ ]:
x = padded_sequences[:,:-1]
labels = padded_sequences[:,-1]

In [ ]:
y = tf.keras.utils.to_categorical(labels,num_classes=vocab_size)

In [ ]:
y.shape

(12307, 8922)

In [ ]:
x.shape

(12307, 19)

In [ ]:
x = x.reshape(x.shape[0],x.shape[1],1)

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding,Bidirectional,LSTM,Dense
from keras.optimizers import Adam

In [ ]:
num_classes = 8922

model = Sequential([
    Embedding(input_dim=12307, output_dim=256, input_length=19),
    Bidirectional(LSTM(256)),
    Dense(num_classes, activation='softmax')
])

optimizer = Adam(lr=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'])


In [ ]:
model.fit(x,y,epochs=50,verbose=1,batch_size=512)

Epoch 1/50
25/25 [==============================] - 39s 2s/step - loss: 2.6777 - acc: 0.4796
Epoch 2/50
25/25 [==============================] - 38s 2s/step - loss: 2.6202 - acc: 0.4839
Epoch 3/50
25/25 [==============================] - 40s 2s/step - loss: 2.5501 - acc: 0.4929
Epoch 4/50
25/25 [==============================] - 39s 2s/step - loss: 2.5224 - acc: 0.4971
Epoch 5/50
25/25 [==============================] - 41s 2s/step - loss: 2.4624 - acc: 0.5036
Epoch 6/50
25/25 [==============================] - 39s 2s/step - loss: 2.3925 - acc: 0.5130
Epoch 7/50
25/25 [==============================] - 39s 2s/step - loss: 2.3337 - acc: 0.5202
Epoch 8/50
25/25 [==============================] - 38s 2s/step - loss: 2.2768 - acc: 0.5321
Epoch 9/50
25/25 [==============================] - 39s 2s/step - loss: 2.2357 - acc: 0.5346
Epoch 10/50
25/25 [==============================] - 39s 2s/step - loss: 2.1695 - acc: 0.5489
Epoch 11/50
25/25 [==============================] - 39s 2s/step - lo

In [ ]:
import numpy as np

seed_text = "I could not help laughing at the ease with which he explained his process of deduction"
next_words = 100

for _ in range(next_words):
    sequence = tokenizer.texts_to_sequences([seed_text])
    padded = pad_sequences(sequence, maxlen=max_seq_len-1)
    predicted = model.predict(padded, verbose=0)
    predicted_index = np.argmax(predicted, axis=-1)
    output_word = ''
    for word, index in tokenizer.word_index.items():
        if index == predicted_index:
            output_word = word
            break
    seed_text += ' ' + output_word
print(seed_text)


I could not help laughing at the ease with which he explained his process of deduction at a smile “g” “g” at the smile “g” “g” known “g” assistance assistance assistance assistance assistance assistance that assistance assistance assistance assistance assistance assistance assistance assistance that the play you you you you you not go to my john so so so so known by “g” “g” known i known i known in him as assistance assistance assistance assistance assistance assistance assistance assistance assistance assistance assistance assistance assistance assistance assistance that the play you you you you you you go to my john so so so so known by “g” “g” known i lent another known known i known


In [ ]:
x